## Motivation {.unnumbered}



In [2]:
## set up environment
suppressMessages({
source("/camp/home/hungm/working/Matthew/library/R/functions/Seurat/processing.R")
source("/camp/home/hungm/working/Matthew/library/R/functions/Seurat/plotting.R")
setwd("/nemo/lab/caladod/working/Matthew/project/matthew/MH_GSE155224")})

## import seurat objects
obj <- qread("seurat/20240826_GSE155224_doublets.qs")

## Calculate Gene Fractions {#sec-calculate_gene_fractions}
  
The function below will perform the calculations of key quality control parameters for every cell but will not filter any cells yet. This is done by calculating the no. of selected reads over the no. of total reads for every cell.

In [ ]:
# run calculate gene fractions if haven't done so already
obj <- calculate_fractions(obj)

In [7]:
obj_list <- SplitObject(obj, split.by = "samples")
obj_list

$filtered_matrix_1
An object of class Seurat 
17895 features across 7502 samples within 3 assays 
Active assay: RNA (17697 features, 2000 variable features)
 3 layers present: scale.data, data, counts
 2 other assays present: HTO, ADT
 3 dimensional reductions calculated: pca, harmony, umap

$filtered_matrix_2
An object of class Seurat 
17895 features across 8227 samples within 3 assays 
Active assay: RNA (17697 features, 2000 variable features)
 3 layers present: scale.data, data, counts
 2 other assays present: HTO, ADT
 3 dimensional reductions calculated: pca, harmony, umap


In [ ]:
for(x in seq_along(obj_list)){
    obj_list[[x]] <- run_decontx(obj_list[[x]], raw.dir = dir[x], cluster = "seurat_clusters")}

In [6]:
run_decontx <- function(x, raw.dir, cluster = NULL, assay = "RNA", feature.type = "Gene Expression"){
    toc <- x[[assay]]$counts
    tod <- Read10X(raw.dir)[[feature.type]]
    output <- decontX(toc, background = tod, z = cluster, delta = c(10, 10), estimateDelta = FALSE)

    ## add contamination score
    score <- data.frame(sc.decontx = output$contamination)
    rownames(score) <- colnames(output$decontXcounts)
    x <- AddMetaData(x, score)

    ## add decontaminated counts
    rna.clean <- as(output$decontXcounts[,colnames(x)], "sparseMatrix")
    x[["DX"]] <- CreateAssay5Object(counts = rna.clean, min.cells = 0, min.features = 0)
    
    return(x)
    }